In [15]:
import math
import logging
FORMAT = '[%(name)s:%(levelname)s]  %(message)s'
logging.basicConfig(level=logging.DEBUG, format=FORMAT)
logger = logging.getLogger('dbg')

def dprint(s):
    logger.debug(s)

def iprint(s):
    logger.info(s)

logger.setLevel(logging.INFO)

In [16]:
class BinarySearchTree:
    def __init__(self, root=None):
        self.root = root

class Node:
    def __init__(self, key, parent=None, left=None, right=None):
        self.key = key
        self.parent = parent
        self.left = left
        self.right = right
    
    def __repr__(self) -> str:
        return f"[{self.key}]"

"    3    "   
"   / \   "    
"  2   5  "
" /   / \ "
"1   4   6"

one = Node(1)
two = Node(2)
three = Node(3)
four = Node(4)
five = Node(5)
six = Node(6)

three.left = two
three.right = five
two.parent = three
two.left = one
one.parent = two

five.parent = three
five.left = four
five.right = six
four.parent = five
six.parent = five

bst = BinarySearchTree(three)

## Binary Search Trees

<img src="media/BST.png" alt="drawing" width="550"/>

BST: for each node $u$: 
- nodes in the _left subtree_ satisfy l.key $\leq$ u.key
- nodes in the _right subtree_ satisfy r.key $\geq$ u.key

**i.e. go left get smaller, go right get bigger**

| Complexity | Average Case       | Worst Case |
| ---------- | ------------       | ---------- |
| Search     | $O(\log n) = O(h)$ | $\Theta(n)$ |
| Insert     | $O(\log n) = O(h)$ | $\Theta(n)$ |
| Delete     | $O(\log n) = O(h)$ | $\Theta(n)$ |


### Tree Height

*Chain:* $h = n - 1$

*Perfect Tree:* $h = \log(n + 1) - 1$

<img src="media/BSTheight.png" alt="drawing" width="750"/>

### Traversal Modes - Depth First Search (DFS)

1. Inorder() - Process node **in-between** vising LST and RST
2. Preorder() - Process node **before** vising LST and RST
3. Postorder() - Process node **after** vising LST and RST

E.g. For the simple binary search tree:

```
    3       
   / \       
  2   5    
 /   / \
1   4   6
```
| Traversal | Path | Complexity |
| - | - | - |
| Inorder()   | 1 -> 2 -> 3 -> 4 -> 5 -> 6 | $\Theta(n)$ |
| Preorder()  | 3 -> 2 -> 1 -> 5 -> 4 -> 6 | $\Theta(n)$ |
| Postorder() | 1 -> 2 -> 4 -> 6 -> 5 -> 3 | $\Theta(n)$ |


In [17]:
def inorder(node):
    if node is not None:
        r_left = inorder(node.left)
        r_right = inorder(node.right)
        return r_left+[node.key]+r_right
    return []

def preorder(node):
    if node is not None:
        r_left = preorder(node.left)
        r_right = preorder(node.right)
        return [node.key]+r_left+r_right
    return []

def postorder(node):
    if node is not None:
        r_left = postorder(node.left)
        r_right = postorder(node.right)
        return r_left+r_right+[node.key]
    return []

print(inorder(bst.root))
print(preorder(bst.root))
print(postorder(bst.root))

[1, 2, 3, 4, 5, 6]
[3, 2, 1, 5, 4, 6]
[1, 2, 4, 6, 5, 3]


### Minimum and Maximum

Simple, as the BST is sorted by design.

In [18]:
def minimum(node: Node):
    while node.left:
        node = node.left
    return node


def maximum(node: Node):
    while node.right:
        node = node.right
    return node

print(f" Min: {minimum(bst.root).key}, Max: {maximum(bst.root).key}")

 Min: 1, Max: 6


### Search - $O(h)$

Search by comparative descent

In [19]:
def search(key: int, node: Node):
    while node and node.key != key:
        if key < node.key:
            node = node.left
        else:
            node = node.right
    return node

print(f" Found: {search(7, bst.root)}")
print(f" Found: {search(6, bst.root)}")

 Found: None
 Found: [6]


### Inorder Predecessor - $O(h)$

Element immediately before node in an inorder traversal.

I.e the maximum value in the left sub-tree, or ascend through parents until the the tree swings to the left.

In [20]:
def predecessor(node: Node):
    if node.left:
        return maximum(node.left)
    else:
        parent = node.parent
        while parent and node != parent.right:
            node = parent
            parent = parent.parent
        return parent
    
print(f"Predecessor of {three} is {predecessor(three)}")
print(f"Predecessor of {four} is {predecessor(four)}")


Predecessor of [3] is [2]
Predecessor of [4] is [3]


### Insert - $O(h)$

This follows the convention in _Cormen_, where BST's allow duplicate trees.

Starting at the root:
- Descend the tree left if new_node.key < node.key else right
- Keep descending **Until a leaf is reached**
- Add the new node under the leaf left of right
- Cover the empty tree case to build a new tree from scratch

In [21]:
def insert(bst: BinarySearchTree, new_node: Node):
    node = bst.root
    parent = None
    while node:
        parent = node
        node = node.left if new_node.key < node.key else node.right
    new_node.parent = parent
    if not parent:  # handle the case when the tree is empty
        bst.root = new_node
    elif new_node.key < parent.key:
        parent.left = new_node
    else:
        parent.right = new_node

tree = BinarySearchTree()
new_keys = [3, 2, 5, 1, 4, 6, 5]
new_nodes = [Node(key) for key in new_keys]
for node in new_nodes:
    insert(tree, node)
print(preorder(tree.root))


[3, 2, 1, 5, 4, 6, 5]


### Delete - $O(h)$

For a node to be deleted, $node$
- Case 1 - $node$ has no right child: shift LST up
- Case 2 - $node$ has no left child: shift RST up
- Case 3 - $node$ has 2 children, (inorder) successor is right child:  upshift successor
- Case 4 - $node$ has 2 children, (inorder) successor is **not** right child: make it so + C3

This process requires a $shift_nodes()$ function, that moves a node and it's subtree to inplace of an old node. This operates through the following routine:
1. if the old node is the root (has no parent), make the new node the root replacing it
2. if the old node is a left child, update its parent to point to the new node
3. if the old node is a right child, update its parent to point to the new node
4. if the new node exists, update its parent pointer


In [22]:
def shift_nodes(bst: BinarySearchTree, old_node: Node, new_node: Node):
    if not old_node.parent:
        bst.root = new_node
    elif old_node == old_node.parent.left:
        old_node.parent.left = new_node
    else:
        old_node.parent.right = new_node
    if new_node:
        new_node.parent = old_node.parent

def delete(bst: BinarySearchTree, node: Node):
    if not node.right:
        # Shift up the left subtree in place
        shift_nodes(bst, node, node.right)
    elif not node.left:
        # Shift up the right subtree in place
        shift_nodes(bst, node, node.right)
    else:
        # find the inorder successor by getting the minimum of the RST of the node
        successor = minimum(node.right)
        if successor != node.right:
            # Detaches the successor and upshifts its RST into its spot
            shift_nodes(bst, successor, successor.right)
            # Place the successor at the top of the RST and shift node.right down one
            successor.right = node.right        # update the successors right pointer
            successor.right.parent = successor  # update he successors right parent 
        # node_successor == node.right
        # detaches the node and upshifts the successor
        shift_nodes(bst, node, successor)
        successor.left = node.left
        successor.left.parent = successor

### Putting it All Together

In [23]:
def main():
    bst = BinarySearchTree()
    insert_keys = [5, 3, 2, 7, 1, 8, 9, 12]
    node_list = [Node(key) for key in insert_keys]
    for node in node_list:
        insert(bst, node)

    # print out traversals
    print(f"Inorder traversal")
    print(inorder(bst.root))
    print("")
    print(f"Preorder traversal")
    print(preorder(bst.root))
    print("")
    print(f"Postorder traversal")
    print(preorder(bst.root))
    print("")

    node_to_delete = node_list[3]
    print(f"Deleting node {node_to_delete}")
    delete(bst, node_to_delete)

    # print out traversal
    print(f"Inorder traversal after deletion")
    print(inorder(bst.root))
    print("")

    # print out minimum and maximum
    print(f"Minimum key: {minimum(bst.root).key}")
    print(f"Maximum key: {maximum(bst.root).key}")

main()


Inorder traversal
[1, 2, 3, 5, 7, 8, 9, 12]

Preorder traversal
[5, 3, 2, 1, 7, 8, 9, 12]

Postorder traversal
[5, 3, 2, 1, 7, 8, 9, 12]

Deleting node [7]
Inorder traversal after deletion
[1, 2, 3, 5, 8, 9, 12]

Minimum key: 1
Maximum key: 12
